# 201017 Overlap component clustering reports

In [1]:
using ProgressMeter
using FilePathsBase; using FilePathsBase: /
using GZip
using JSON
using Mmap
using DataFrames
using CSV
using StatsBase
using CategoricalArrays
using HDF5
using Markdown
using DataStructures: counter
using AbstractTrees
using Clustering
using Dates

In [2]:
using PlotlyJS
using Colors
using ColorSchemes

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1322052592698447000\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-7920146709185753207\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1322052592698447000\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
using Revise

using Midas
using Midas.Pairwise: npairs

using ClusterAnalysis
using ClusterAnalysis: leaf_class_counts, find_strong_clusters, hclust_node_dists, subtree_classes

using MidasPlots
using MidasPlots: Dendrogram, dgleaf_edges, ClassValues
using MidasPlots.Plotly: PlotlyDendrogram, subplot_axes, subplot_axes!, gettraces, setaxes!, vector_attrs!, class_color_strip, make_colorscale

using TriMatrices

## Func defs

In [4]:
findclass(a::CategoricalArray, cls::CategoricalValue) = findall(==(cls), a)
findclass(a::CategoricalArray, i::Integer) = findclass(a, a.pool[i])
findclass(a::CategoricalArray, clss::AbstractVector{<:CategoricalValue}) = findall(∈(clss), a)
findclass(a::CategoricalArray, is::AbstractVector{<:Integer}) = findall(∈(is), a.refs)
selectclass(a::AbstractVector, c::CategoricalArray, cls) = a[findclass(c, cls)]

selectclass (generic function with 1 method)

## File paths

In [5]:
infiles = Dict(
    :taxonomy => p"/home/jared/notebooks/midas/midas-notebooks-2019/build-v1-database/out/3-curated-taxonomy-assignments.csv",
    :distances => p"../../data/intermediate/200727-find-overlaps/genome-pw-distances.raw-float32",
    :overlaps => p"../../data/intermediate/200727-find-overlaps/200729-refseq-curated-1.1_beta-species-overlaps.h5",
);

In [6]:
out_dirs = Dict(
    :intermediate => p"../../data/intermediate/201013-overlap-clustering-analysis/201017-overlap-component-clustering-reports/",
    :reports => p"../../reports/201013-overlap-clustering-analysis/201017-overlap-component-clustering-reports/",
)

for d in values(out_dirs)
    isdir(d) || mkpath(d)
end

## Load data

### Taxonomy

In [7]:
taxdf = DataFrame(CSV.File(infiles[:taxonomy]));

In [8]:
sig_genera = categorical(taxdf[!, :genus])
genera = levels(sig_genera)
ngenera = length(genera)

sig_species = categorical([(row[:genus], row[:species]) for row in eachrow(taxdf)])
species = levels(sig_species)
speciesnames = [join(s, " ") for s in species]
nspecies = length(species)

ngenera, nspecies

(419, 1438)

In [9]:
genus_counts = counts(sig_genera.refs)
species_counts = counts(sig_species.refs)
;

In [10]:
genome_accs = [last(split(k, "/")) for k in taxdf[!, :key]];

In [11]:
species_to_genus = [findfirst(==(genus), genera) for (genus, spname) in species];

In [12]:
nsigs = size(taxdf, 1)

50752

### Distances

In [13]:
pw_data = Mmap.mmap(open(infiles[:distances], "r"), Vector{Float32}, (npairs(nsigs),));

pw_dists = TriMatrix(TriSymmetric{false}(), nsigs, pw_data);

In [14]:
@assert !any(<(0), pw_data)

### Overlaps

In [15]:
h5open(string(infiles[:overlaps])) do h5
    @assert read(h5, "genome_keys") == taxdf[!, :key]

    global min_inter = collect(zip(read(h5, "min_inter_indices"), read(h5, "min_inter_dists")))
    global max_intra = collect(zip(read(h5, "max_intra_indices"), read(h5, "max_intra_dists")))

    global has_overlap = read(h5, "has_overlap")
    global overlap_components_vec = read(h5, "overlap_components")
end;

In [16]:
overlap_pairs = [i => min_inter[i][1] for i in has_overlap];

In [17]:
overlap_components = [findall(==(i), overlap_components_vec) for i in 1:maximum(overlap_components_vec)];

In [18]:
overlap_edges = counter(
    Int(sig_species.refs[i]) => Int(sig_species.refs[j])
    for (i, j) in overlap_pairs
);

## Plot and reporting funcs

In [19]:
pal = collect(colorschemes[:tableau_20].colors)
pal = pal[vcat(1:2:17, 2:2:18)]
classcolors = ClassValues(pal, Gray(.75))

In [20]:
struct Namespace
    _fields::Dict{Symbol, Any}
end
    
Namespace(; kw...) = Namespace(Dict(kw))

function Base.getproperty(ns::Namespace, p::Symbol)
    f = getfield(ns, :_fields)
    p == :_fields && return f
    try
        return f[p]
    catch KeyError
        error("Namespace has no field $p")
    end
end

function Base.setproperty!(ns::Namespace, p::Symbol, v)
    f = getfield(ns, :_fields)
    f[p] = v
end

macro unpack_ns(e, symbols::Symbol...)
    x = [Expr(:., :ns, QuoteNode(s)) for s in symbols]
    rhs = quote
        let ns = $(esc(e))
            $(Expr(:tuple, x...))
        end
    end
    return Expr(:(=), Expr(:tuple, esc.(symbols)...), rhs)
end

@unpack_ns (macro with 1 method)

In [21]:
function componentdata(i::Int)
    i = i
    comp = overlap_components[i]
    nspecies = length(comp)
    idxs = [findclass(sig_species, c) for c in comp]
    idxs_flat = vcat(idxs...)
    genera = unique(g for (g, s) in species[comp])
    singlegenus = length(genera) == 1
    
    if singlegenus
        genus = only(genera)
        groupnames = [s for (g, s) in species[comp]]
    else
       genus = nothing
       groupnames = speciesnames[comp]
    end
    
    classnames = ClassValues(groupnames, "(mixed)");
    ngenomes = length(idxs_flat)
    genome_labels = [l for (l, n) in enumerate(species_counts[comp]) for _ in 1:n];
    dmat = pw_dists[idxs_flat, idxs_flat];
    
    return Namespace(;
        i, species=comp, idxs, idxs_flat, genera, singlegenus,
        genus, groupnames, classnames, ngenomes, nspecies,
        genome_labels, dmat
    )
end

componentdata (generic function with 1 method)

In [22]:
function do_clustering(comp)
    # Cluster
    hc = hclust(comp.dmat, linkage=:complete)
    tree = HClustTree(hc)

    # Find optimal cut
    strong_clusters = find_strong_clusters(tree, comp.dmat);
    node_classes = subtree_classes(tree, comp.genome_labels, with_leaves=true);
    cut = makecut(node -> node.id ∈ strong_clusters && node_classes[node.id] != 0, tree);
    
    # Coarsen cut if needed
    truncated_from = nothing
    max_leaves = 40
    
    if length(cut) > max_leaves
        truncated_from = length(cut)
        cut = coarsen_cut(tree, cut, k=max_leaves, accept=node -> node_classes[node.id] != 0);
    end
    
    dg = Dendrogram(tree, trunc=node -> node.id ∈ cut)
    leaf_classes = [node_classes[id] for id in dg.leaves]
    classnames = ClassValues(comp.groupnames, "(mixed)")
    
    return Namespace(;
        tree, dg, cut, strong_clusters, node_classes, truncated_from, leaf_classes,
        classnames,
    )
end

do_clustering (generic function with 1 method)

In [23]:
function component_species_df(comp)
    DataFrame([(genus=species[i][1], species=species[i][2], ngenomes=species_counts[i]) for i in comp.species])
end

function component_edges_df(comp)
    df = DataFrame([String, String, Int, Float64], [:A, :B, :C, :D])
    
    for ((i, j), c) in overlap_edges
        if i ∈ comp.species
            push!(df, (
                speciesnames[i],
                speciesnames[j],
                c,
                c / (species_counts[i] * species_counts[j])
            ))
        end
    end
    
    sort!(df, [:A, :B])
    rename!(df, ["Intra Species", "Inter Species", "Overlapping Pairs", "Overlap fraction"])
    
    return df
end

component_edges_df (generic function with 1 method)

In [24]:
function species_colors_html(comp)
    divs = String[]
    
    for (i, name) in enumerate(comp.groupnames)
        color = classcolors[i]
        bc = hex(color)
        bg = hex(RGBA(color, .5), :RRGGBBAA)
        style = "border-color: #$bc; background-color: #$bg;"
        d = "<div style=\"$style\">$name</div>"
        push!(divs, d)
    end
    
    return join(divs, "\n")
end

species_colors_html (generic function with 1 method)

In [25]:
REPORT_CSS = """

body {
    padding: 20px;
    font-family: sans-serif;
}

h1 {
    margin: 40px 0;
}

h2 {
    margin-top: 40px;
    margin-bottom: 25px;
}

table {
    border-collapse: collapse;
    margin-bottom: 15px;
}

td, th {
    border: 1px solid black;
    padding: 4px .5em;
    text-align: left;
}

div.species-colors > div {
    display: inline-block;
    border: 2px solid;
    margin: 10px;
    padding: 5px 10px;
}

div.plot {
    margin: 50px 0;
}

dl dt {
    font-weight: bold;
}
""";

In [26]:
report_template(;
    title,
    header,
    table1_html,
    table2_html,
    plt1,
    plt2,
    species_colors_html,
    plot_msg,
    timestamp,
    ) = """
    <html>
        <head>
            <title>$title</title>

            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

            <script>
                document.addEventListener("DOMContentLoaded", function(){
                    var plt1_json=$(json(plt1));
                    var plt2_json=$(json(plt2));

                    var plt1_elem = document.getElementById("plot1");
                    var plt2_elem = document.getElementById("plot2");

                    Plotly.newPlot(plt1_elem, plt1_json);
                    Plotly.newPlot(plt2_elem, plt2_json);

                    window.onresize = function() {
                        Plotly.Plots.resize(plt1_elem);
                        Plotly.Plots.resize(plt2_elem);
                    };
                });

            </script>

            <style>$REPORT_CSS</style>
        </head>

        <body>
            <h1>$header</h1>

            <dl>
                <dt>Experiment:</dt>
                <dd><code>201017-overlap-component-reports</code></dd>
                <dt>Generated:</dt>
                <dd>$timestamp</dd>
            </dl>

            <h2>Species</h2>
            $table1_html

            <h2>Overlaps</h2>
            $table2_html

            <h2>Plots</h2>
            $plot_msg

            <div class="species-colors">$species_colors_html</div>

            <div class="plot" id="plot1"></div>
            <div class="plot" id="plot2"></div>
        </body>
    </html>
"""

report_template (generic function with 1 method)

In [27]:
function component_report(comp, clust, plt1, plt2)
    kw = Dict{Symbol, Any}(
        :title => "Component report $(comp.i)",
        :plt1 => plt1,
        :plt2 => plt2,
        :species_colors_html => species_colors_html(comp),
        :timestamp => Dates.format(Dates.now(), "yymmdd H:M"),
    )
    
    kw[:header] =  string(
        "Component $(comp.i): ",
        join(comp.genera, ", "),
        " ($(comp.nspecies) species, $(comp.ngenomes) genomes)"
    )
    
    if isnothing(clust.truncated_from)
        kw[:plot_msg] = ""
    else
        nleaves = length(clust.dg.leaves)
        kw[:plot_msg] = "<p>Optimal truncation of dendrogram had $(clust.truncated_from) leaves, coarsened to $(nleaves) to improve readability.</p>"
    end
    
    buf = IOBuffer()
    
    sp_df = component_species_df(comp)
    show(buf, MIME"text/html"(), sp_df, eltypes=false, summary=false)
    kw[:table1_html] = String(take!(buf))
    
    e_df = component_edges_df(comp)
    show(buf, MIME"text/html"(), e_df, eltypes=false, summary=false)
    kw[:table2_html] = String(take!(buf))
        
    return report_template(; kw...)
end

component_report (generic function with 1 method)

In [28]:
function plotly_dendrogram(clust, dgattr=attr(); horizontal::Bool=false)
    @unpack_ns clust dg tree node_classes strong_clusters classnames
    
    pdg = PlotlyDendrogram(dg, dgattr, horizontal=horizontal)
    
    # Node trace attributes
    vector_attrs!(pdg.nodes_trace, pdg.node_ids) do id
        node = tree.nodes[id]
        c = node_classes[id]
        return Dict(
            :marker_size => id ∈ strong_clusters ? 8 : 4,
            :marker_color => classcolors[c],
            :text => string(id),
            :customdata => [
                node.count,
                classnames[c],
                string(id ∈ strong_clusters ? "Strong" : "Weak", " cluster"),
            ],
        )
    end
    
    yax = horizontal ? "x" : "y"
    merge!(pdg.nodes_trace, attr(
        hovertemplate="""
        <b>%{text}</b><br>
        %{$yax:.3f}<br>
        %{customdata[0]} genomes<br>
        %{customdata[1]}<br>
        %{customdata[2]}
        <extra></extra>
        """,
    ))
    
    # Branch trace colors
    for (id, trace) in pdg.branch_traces
        trace[:line_color] = classcolors[node_classes[id]]
    end
    
    return pdg
end

plotly_dendrogram (generic function with 2 methods)

In [29]:
function leaf_color_strip(clust; vertical::Bool=false, kw...)
    @unpack_ns clust dg tree leaf_classes classnames
    
    edges = dgleaf_edges(dg)
    
    return class_color_strip(
        leaf_classes,
        classcolors;
        vertical=vertical,
        edges=edges,
        text=[id <= tree.n ? "Genome $id" : "Cluster $id" for id in dg.leaves],
        customdata=[[classnames[c]] for c in leaf_classes],
        hovertemplate="""
        <b>%{text}</b><br>
        %{customdata[0]}
        <extra></extra>
        """,
        gap=1,
        kw...
    )
end

leaf_color_strip (generic function with 1 method)

In [30]:
function big_dendrogram(clust)
    # Y axes
    yaxes = subplot_axes!(
        [
            attr(visible=false),
            attr(rangemode=:nonnegative),
        ],
        [.1, 1.0],
        sep=0.02
    )
    
    # Dendrogram
    pdg = plotly_dendrogram(clust, attr(yaxis="y2"))
    
    # Labels
    labels_t = leaf_color_strip(clust)
    
    layout = Layout(
        xaxis=attr(
            visible=false,
        ),
        showlegend=false,
        hovermode=:closest,
    )
    setaxes!(layout, :y, yaxes)
    
    return plot(vcat(gettraces(pdg), [labels_t]), layout)
end

big_dendrogram (generic function with 1 method)

In [31]:
function clustermap(comp, clust)
    @unpack_ns clust tree dg leaf_classes classnames
    
    # X axes
    xaxes = subplot_axes!(
        [
            attr(
                rangemode=:nonnegative,
                autorange=:reversed,
            ),
            attr(
                visible=false,
            ),
            attr(
                visible=false,
                scaleanchor=:y,
            ),
        ],
        [.4, .05, 1.],
        sep=.01,
    )
    
    # Dendrogram
    pdg = plotly_dendrogram(clust, horizontal=true)
    
    # Labels
    labels_t = leaf_color_strip(clust, xaxis="x2", vertical=true)
    
    # Heatmap
    edges = dgleaf_edges(dg)
    leaf_names = [id <= tree.n ? "Genome $id" : "Cluster $id" for id in dg.leaves]
    hm_leaf_text = [string(n, ": ", classnames[c]) for (n, c) in zip(leaf_names, leaf_classes)]
    
    hm_t = heatmap(
        z=hclust_node_dists(tree, comp.dmat, dg.leaves),
        x=edges,
        y=edges,
        customdata=[[t1, t2] for t1 in hm_leaf_text, t2 in hm_leaf_text],
        hovertemplate="""
        %{customdata[0]}<br>
        %{customdata[1]}<br>
        %{z:.3f}
        <extra></extra>
        """,
        colorscale=make_colorscale(:magma),
        xaxis="x3",
    )
    
    layout = Layout(
        yaxis=attr(
            visible=false,
        ),
        showlegend=false,
        hovermode=:closest,
        width=1000,
        height=800,
    )
    setaxes!(layout, :x, xaxes)
    
    return plot(vcat(gettraces(pdg), [hm_t, labels_t]), layout)
end

clustermap (generic function with 1 method)

## Generate reports

In [32]:
@showprogress for ci in 1:length(overlap_components)
    comp = componentdata(ci)
    
    # Skip very large components
    max_comp_size = 15
    if comp.nspecies > max_comp_size
        println("Component $(comp.i) has size $(comp.nspecies), skipping")
        continue
    end
    
    # Clustering
    clust = do_clustering(comp)
    
    # Plots
    plt1 = big_dendrogram(clust)
    plt2 = clustermap(comp, clust)

    # Make report
    report_html = component_report(comp, clust, plt1, plt2)

    # Save report
    open(out_dirs[:reports] / "component-$ci.html", "w") do f
        write(f, report_html)
    end
    
    # Save tree and genomes to cross ref with genome/cluster #s in report
    open(out_dirs[:intermediate] / "genomes-$ci.json", "w") do f
        JSON.print(f, genome_accs[comp.idxs_flat])
    end
    
    open(out_dirs[:intermediate] / "tree-$ci.json", "w") do f
        JSON.print(f, clust.tree)
    end
end

Progress:  17%|███████                                  |  ETA: 0:03:03

Component 12 has size 22, skipping


Progress: 100%|█████████████████████████████████████████| Time: 0:00:40
